In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))

# Web Scraping

In [134]:
# Importando bibliotecas:
import pandas as pd
from urllib.request import Request, urlopen, urlretrieve
from urllib.error import URLError, HTTPError
from bs4 import BeautifulSoup

# Declarando variável cards:
cards = []

## Obtendo o HTML e o total de páginas:
url = 'https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/aluguel/apartamentos?o=1'
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
try:
    req = Request(url, headers=headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    
except HTTPError as e:
    print(e.status, e.reason)
    
except URLError as e:
    print(e.reason)

# Criando o objeto soup:
soup = BeautifulSoup(html, 'html.parser')

# Número de páginas: *dependendo do site pode ser atomatizado ou coletado manualmente como nesse caso
pages = int(29)

## Iterando por todas as páginas do site:
for i in range(pages):
    ## Obtendo o HTML
    headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.105 Safari/537.36'}
    req = Request('https://rn.olx.com.br/rio-grande-do-norte/natal/imoveis/aluguel/apartamentos?o=' + str(i + 1), headers=headers)
    response = urlopen(req)
    html = response.read().decode('utf-8')
    soup = BeautifulSoup(html, 'html.parser')

    # Obtendo as TAGs de interesse
    anuncios = soup.find('div', {"id": "column-main-content"}).findAll('div', class_="fnmrjs-2 jiSLYe")
    # Coletando as informações dos CARDS
    for anuncio in anuncios:
        card = {}
        
        # Nome do anúncio
        nome = anuncio.find("h2").getText()
        card['NOME'] = nome
        
        # Valor
        span_valor = anuncio.find("span", class_="sc-ifAKCX eoKYee")
        valor = span_valor.get_text() if span_valor else ''
        card["VALOR"] = valor

        # Detalhes
        span_detalhe = anuncio.find("span", class_="sc-1j5op1p-0 lnqdIU sc-ifAKCX eLPYJb")
        detalhe = span_detalhe.get_text().split('|') if span_detalhe else ''
        card['DETALHES'] = detalhe

        # Endereço
        endereco = anuncio.find('span', class_="sc-7l84qu-1 ciykCV sc-ifAKCX hGjdHN").getText().split(', ')[1]
        card['ENDEREÇO'] = endereco

        # Adicionando resultado a lista cards
        cards.append(card)

# Criando um DataFrame com os resultados
df = pd.DataFrame(cards)
df

,NOME,VALOR,DETALHES,ENDEREÇO
0,Alugo apartamento em condomínio Portamaris com...,R$ 4.000,"[3 quartos , 79m² , 2 vagas]",Ponta Negra
1,Apto tipo suite e apto de 01 e 02 quartos em c...,R$ 380,[0m²],Cidade Alta
2,Apartamento alto padrão,R$ 2.500,"[3 quartos , 118m² , Condomínio: R$ 0 , 2 v...",Candelária
3,Lindo apto locação anual,R$ 1.750,"[1 quarto , 43m² , 1 vaga]",Capim Macio
4,"Apartamento com 2 dormitórios para alugar, 50 ...",R$ 2.100,"[2 quartos , 50m²]",Petrópolis
...,...,...,...,...
1438,Kitnet ponta negra,,"[1 quarto , 111111m² , Condomínio: R$ 0]",Ponta Negra
1439,Alugo no Sun Towers - Cidade Satélite,R$ 950,"[2 quartos , 55m² , Condomínio: R$ 350 , 1 ...",Pitimbu
1440,Apartamento Cidade Alta,R$ 650,"[1 quarto , 32m² , 1 vaga]",Ribeira
1441,Kitnet UFRN cidade jardim capim macio,R$ 800,,Capim Macio
